# Notes

- Maybe generating a "global" Fab model with AF2 and grafting a ABlooper predicted CDR to it?
    - Class: **pyrosetta.rosetta.protocols.antibody.AntibodyCDRGrafter**

# Workflow

**input**: PDBx2 + ms2.mzML

**output**: ...

1. Clean PDB files and merge
2. ...

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

from pyrosetta import *
from pyrosetta.rosetta.core.scoring import *
from pyrosetta.rosetta.core import select
from pyrosetta.rosetta.core.select import residue_selector as selections
from pyrosetta.rosetta import protocols
from pyrosetta.rosetta.core.pack.task import operation
from pyrosetta.rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from pyrosetta.rosetta.protocols.antibody import *
from pyrosetta.toolbox import *

import py3Dmol

In [ ]:
def plot_plddt_legend(dpi=100):
  thresh = ['plDDT:', 'Very low (<50)', 'Low (60)',
            'OK (70)', 'Confident (80)', 'Very high (>90)']
  plt.figure(figsize=(1, 0.1), dpi=dpi)
  ########################################
  for c in ["#FFFFFF", "#FF0000", "#FFFF00", "#00FF00", "#00FFFF", "#0000FF"]:
    plt.bar(0, 0, color=c)
  plt.legend(thresh, frameon=False,
             loc='center', ncol=6,
             handletextpad=1,
             columnspacing=1,
             markerscale=0.5,)
  plt.axis(False)
  return plt

def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  view.addModel(open(clean_pdb_file, 'r').read(), 'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop': 'b',
                  'gradient': 'roygb', 'min': 50, 'max': 90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color': 'spectrum'}})
  elif color == "chain":
    chains = len(queries[0][1]) + 1 if is_complex else 1
    for n, chain, color in zip(range(chains), list("ABCDEFGH"),
                               ["lime", "cyan", "magenta", "yellow", "salmon", "white", "blue", "orange"]):
      view.setStyle({'chain': chain}, {'cartoon': {'color': color}})
  if show_sidechains:
    BB = ['C', 'O', 'N']
    view.addStyle({'and': [{'resn': ["GLY", "PRO"], 'invert':True}, {'atom': BB, 'invert': True}]},
                  {'stick': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})
    view.addStyle({'and': [{'resn': "GLY"}, {'atom': 'CA'}]},
                  {'sphere': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})
    view.addStyle({'and': [{'resn': "PRO"}, {'atom': ['C', 'O'], 'invert':True}]},
                  {'stick': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})
  if show_mainchains:
    BB = ['C', 'O', 'N', 'CA']
    view.addStyle(
        {'atom': BB}, {'stick': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})

  view.setBackgroundColor('lightgray')
  view.zoomTo()
  return view

In [ ]:
init('-use_input_sc -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -no_fconfig')


In [ ]:
pdb_file = './data/top01_fab.pdb'
clean_pdb_file = './data/top01_fab_clean.pdb'
cleanATOM(pdb_file=pdb_file, out_file=clean_pdb_file)

pose = pose_from_pdb(clean_pdb_file)
original_pose = pose.clone()

In [ ]:
pdb_info = pose.pdb_info()
nres = pose.total_residue()
seq = pose.sequence()

line_break = 50
seq_print = ''.join(
    [
        c
        if (i+1)%(line_break+1)!=0
        else '\n'
        for i, c in enumerate(seq)
    ]
)

print(pdb_info)
print(f'Residues [n={nres}]\n{seq_print}')

In [ ]:
pdb_nums = [
    (str(pdb_info.number(i)) + pdb_info.icode(i)).strip()
    for i in range(1, nres + 1)
]
chains = [pdb_info.chain(i) for i in range(1, nres + 1)]
unique_chains = set(chains)

phis = [pose.phi(i) for i in range(1, nres + 1)]
psis = [pose.psi(i) for i in range(1, nres + 1)]
omegas = [pose.omega(i) for i in range(1, nres + 1)]

sfxn = get_score_function()
sfxn(pose)

DSSP = protocols.moves.DsspMover()
DSSP.apply(pose)
ss = pose.secstruct()

In [ ]:
# TODO:
# - Change to f-strings
print(f'Secondary Structure:\n{ss}')
print('\t' + str(100. * ss.count('H') / len(ss))[:4] + '% Helical (H)')
print('\t' + str(100. * ss.count('E') / len(ss))[:4] + '% Sheet (E)')
print('\t' + str(100. * ss.count('L') / len(ss))[:4] + '% Loop (L)')

In [ ]:
is_complex = False
rank_num = 1
show_sidechains = False
show_mainchains = False
color = 'lDDT'


if color == "lDDT":
  plot_plddt_legend(dpi=77.8).show()

show_pdb(rank_num, show_sidechains, show_mainchains, color).show()

In [ ]:
energies = pose.energies()
residue_energies = [
    energies.residue_total_energy(i)
    for i in range(1, nres + 1)
]

pose_hbonds = hbonds.HBondSet()

hbonds.fill_hbond_set(pose, False, pose_hbonds)

hbond_dictionary = {}

for residue in range(1, nres + 1):
    hbond_text = ''
    for hbond in range(1, pose_hbonds.nhbonds() + 1):
        hbond = pose_hbonds.hbond(hbond)
        acceptor_residue = hbond.acc_res()
        donor_residue = hbond.don_res()
        if residue == acceptor_residue or residue == donor_residue:
            # Rewrite to save as data structure and not string!
            # And skip duplicates.
            hbond_text += str(donor_residue).ljust(4) + ' ' + \
                str(pose.residue(donor_residue).atom_name(
                    hbond.don_hatm())).strip().ljust(4) + \
                ' => ' + str(acceptor_residue).ljust(4) + ' ' + \
                str(pose.residue(acceptor_residue).atom_name(
                    hbond.acc_atm())).strip().ljust(4) + \
                ' |score: ' + str(hbond.energy()) + '\n'
    hbond_dictionary[residue] = hbond_text if len(hbond_text)>1 else None

In [ ]:
print('(donor residue) (donor atom) => (acceptor residue) (acceptor atom) |score')

for k, v in hbond_dictionary.items():
    if v:
        print(v)

In [ ]:
ab_info = protocols.antibody.AntibodyInfo(
    pose, protocols.antibody.Kabat_Scheme, protocols.antibody.North)

enum_manager = protocols.antibody.AntibodyEnumManager()

In [ ]:
list_dict = {chain: [] for chain in unique_chains}

view = py3Dmol.view()
view.addModel(open('./data/top01_fab_clean.pdb', 'r').read(), 'pdb')
view.setBackgroundColor('lightgrey')

for i in range(1, len(seq)+1):
    chain = pose.pdb_info().chain(i)
    residue = pose.pdb_info().number(i)

    list_dict[chain].append('')

view.zoomTo()

In [ ]:
import nglview as nv

color_list = []

for i in range(1, pose.size()+1):
    region = enum_manager.antibody_region_enum_to_string(
        ab_info.get_region_of_residue(pose, i)
    )
    if region == 'framework_region':
        tmp_list = ['black', str(i)]
    else:
        cdr = enum_manager.cdr_name_enum_to_string(
            ab_info.get_CDRNameEnum_of_residue(pose, i))
        if cdr.startswith('L'):
            tmp_list = ['red', str(i)]
        elif cdr.startswith('H'):
            tmp_list = ['blue', str(i)]
        else:
            tmp_list = ['white', str(i)]
    color_list.append(tmp_list)

color_scheme = nv.color._ColorScheme(color_list, 0)

v = nv.show_rosetta(pose, default_representation=False)

v.clear()
v.add_cartoon('protein', color=color_scheme)
v

In [ ]:
# nbr_selector = selections.NeighborhoodResidueSelector()
# cdr_selector = protocols.antibody.residue_selector.CDRResidueSelector()
# cdr_selector.set_cdr(h1)

# nbr_selector.set_focus_selector(cdr_selector)
# nbr_selector.set_include_focus_in_subset(True)

# prevent_repacking_rlt = operation.PreventRepackingRLT()

# prevent_subset_repacking = operation.OperateOnResidueSubset(
#     prevent_repacking_rlt,
#     nbr_selector,
#     True
# )


In [ ]:
# cdr_res = []
# for i in select.get_residue_set_from_subset(cdr_selector.apply(pose)):
#     #print(i)
#     cdr_res.append(i)